In [9]:
%load_ext autoreload
%autoreload 2
import sys
is_LSTM = False
is_plus = [True, False][1]
out_part = ['', 'clf_out', 'no_out_flg'][2]
set_no = [0,1,2,3][3]
const_cnt = 2
is_oof = [True, False][1]
import gc
import re
import pandas as pd
import numpy as np
import os
import time
import datetime
import glob
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

#========================================================================
# Keras 
# Corporación Favorita Grocery Sales Forecasting
sys.path.append(f'{HOME}/kaggle/data_analysis/model')
from nn_keras import mercari_1st_NN, corp_1st_LSTM, RMSE
from keras import callbacks
from keras import optimizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#========================================================================

#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg', 'clf_pred']
stack_name='keras'
model_type='keras'
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
seed = 328
#========================================================================


#========================================================================
# Data Load 
print("Preparing dataset...")
# win_path = f'../features/4_winner/*.gz'
# Ensemble 1
set1 = f'../model/E1_set/*.gz'
# Ensemble 2
set2 = f'../model/E2_set/*.gz'
# Ensemble 3
set3 = f'../model/E3_set/*.gz'
# Ensemble 4
set4 = f'../model/E4_set/*.gz'

set_list = [set1, set2, set3, set4]
win_path = set_list[set_no]

win_path_list = glob.glob(win_path)

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target, 'first_active_month', 'no_out_flg']]
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
feature_list = utils.parallel_load_data(path_list=win_path_list)
df = pd.concat(feature_list, axis=1)
train = pd.concat([base_train, df.iloc[:len(base_train), :]], axis=1)
test = pd.concat([base_test, df.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)

train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True , drop=True)
use_cols = [col for col in train.columns if col not in ignore_list]
train_ids = train[key].values
df_stack = train[[key, target]].copy().set_index(key)

if out_part=='clf_out':
#     train = train[train[target]>-30]
    base_clf = utils.read_pkl_gzip('../input/base_clf.gz')[[key, 'clf_pred']]
    train = train.merge(base_clf, on=key, how='inner')
    train = train[train['clf_pred']<0.01]
    test = test.merge(base_clf, on=key, how='inner')
    test = test[test['clf_pred']<0.01]
    
    train.drop('clf_pred', axis=1, inplace=True)
    test.drop('clf_pred', axis=1, inplace=True)
elif out_part=='no_out_flg':
    train = train[train['no_out_flg']==1]
    test = test[test['no_out_flg']==1]
    train.drop('no_out_flg', axis=1, inplace=True)
    test.drop('no_out_flg', axis=1, inplace=True)
#========================================================================

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Preparing dataset...


In [10]:
#========================================================================
# 正規化の前処理(Null埋め, inf, -infの処理) 
for col in train.columns:
    if col in ignore_list: continue
        
    train[col] = impute_feature(train, col)
    test[col] = impute_feature(test, col)

while True:
    inf_list = []
    for col in use_cols:
        tmp = (train[col]==np.inf).sum()
        tmp2 = (train[col]==-np.inf).sum()
        if tmp>0:
            inf_list.append(col)
        if tmp2>0:
            inf_list.append(col)
        
    for col in inf_list:
        print("Ramain inf...")
        train[col] = impute_feature(train, col)
        test[col] = impute_feature(test, col)
    if len(inf_list)==0:
        break
#========================================================================

In [11]:
#========================================================================
# Construction Value Range Preprocessing
def constraction(feature, is_viz=False, out_range=1.64):
    if is_viz:
        print('before:', feature.max(), feature.min())
    std = feature.std()
    avg = feature.mean()
    z_val = (feature - avg)/std
    
    # Pass the Case No Outlier
    try:
        p_min = feature[feature>=out_range].min()
        feature = np.where(z_val>=out_range, p_min, feature)
    except ValueError:
        pass
    
    try:
        m_max = feature[feature<=-1*out_range].max()
        feature = np.where(z_val<=-1*out_range, m_max, feature)
    except ValueError:
        pass
    
    if is_viz:
        print('after:', feature.max(), feature.min())   
    return feature

train_test = pd.concat([train, test], axis=0)
for col in use_cols:
    feature = train_test[col].values
    
    for i in range(const_cnt):
        feature = constraction(feature)
    
    feature = feature.astype('float32')
    train_test[col] = feature
train = train_test[~train_test[target].isnull()]
test = train_test[train_test[target].isnull()]

scaler = StandardScaler()
scaler.fit(pd.concat([train[use_cols], test[use_cols]]))   
x_test = scaler.transform(test[use_cols])
Y = train[target]

# ========================================================================


#========================================================================
# CVの準備
fold = 6
if out_part == 'all':
    kfold = utils.read_pkl_gzip('../input/ods_kfold.gz')
elif out_part=='clf_out':
    kfold = utils.read_pkl_gzip('../input/ods_clf001_thres_kfold.gz')
elif out_part=='no_out_flg':
    kfold = utils.read_pkl_gzip('../input/ods_no_out_flg_kfold.gz')
if is_plus:
    y_min = Y.min()
    Y = Y - (y_min-1)

# x_test = x_test.as_matrix()
# For LSTM
if is_LSTM:
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
    
print(f"Train: {train.shape} | Test: {test.shape}") 
#========================================================================

Train: (40858, 293) | Test: (24198, 293)


In [13]:
%load_ext autoreload
%autoreload 2
from nn_keras import mercari_1st_NN, corp_1st_LSTM
#========================================================================
# NN Model Setting 
N_EPOCHS = 15
N_EPOCHS = 30
# N_EPOCHS = 10
# learning_rate = 1e-5
# learning_rate = 1e-4
learning_rate = 1e-3
# lerning_rate = 3e-3

break_cnt=0
first_batch=6 # 7: 128
# first_batch=7 # 7: 128

if is_LSTM:
    model = corp_1st_LSTM(input_rows=1, input_cols=len(use_cols))
else:
    model = mercari_1st_NN(input_cols=len(use_cols))

opt = optimizers.Adam(lr=learning_rate)
model.compile(loss=RMSE, optimizer=opt, metrics=[RMSE])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
]
#========================================================================

#========================================================================
# Result Box
model_list = []
result_list = []
score_list = []
val_pred_list = []
test_pred = np.zeros(len(test))
#========================================================================

#========================================================================
# Train & Prediction Start

for fold_no, (trn_idx, val_idx) in enumerate(zip(*kfold)):

    #========================================================================
    # Make Dataset
    X_train, y_train = train.loc[train[key].isin(trn_idx), :][use_cols], Y.loc[train[key].isin(trn_idx)]
    X_val, y_val = train.loc[train[key].isin(val_idx), :][use_cols], Y.loc[train[key].isin(val_idx)]
    
    X_train[:] = scaler.transform(X_train)
    X_val[:] = scaler.transform(X_val)
    X_train = X_train.as_matrix()
    X_val = X_val.as_matrix()
    
    if is_LSTM:
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    #========================================================================
    
    cnt = -1
    while True:
        cnt+=1
        # Fitting
        # なぜか平均を引いてる？そのほうがfitするの？
        # model.fit(X_train, y- y_mean, batch_size = batch_size, epochs = N_EPOCHS, verbose=2,
        #            validation_data=(X_val, y_val - y_mean), callbacks=callbacks )
    #     model.fit(X_train, y_train, batch_size = batch_size, epochs = N_EPOCHS, verbose=2,
    #                validation_data=(X_val, y_val), callbacks=callbacks )
        
        batch_size = 2**(first_batch + cnt)
        model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val)
                  , batch_size=batch_size, epochs=N_EPOCHS
                  , verbose=2, callbacks=callbacks)

        # Prediction
        if is_oof:
            val_id_list = list(set(train_ids) - set(trn_idx))
            X_val = train.loc[train[key].isin(val_id_list), :][use_cols]
            y_val = Y.loc[train[key].isin(val_id_list)]
            y_pred = model.predict(X_val)
            tmp_val = train.loc[train[key].isin(val_id_list), :][[key, target]].set_index(key)
            tmp_val['prediction'] = y_pred
            df_stack['pred_{fold_no}'] = tmp_val['prediction']
            del tmp_val
            gc.collect()
        else:
            y_pred = model.predict(X_val)
        out_cnt = (y_pred==np.inf).sum()+(y_pred==-np.inf).sum()+(y_pred!=y_pred).sum()
        
        if out_cnt>0:
            print("Exist Inf or NaN")
            sys.exit()
            
        break
    
    y_pred = y_pred.reshape(y_pred.shape[0], )
    tmp_pred = model.predict(x_test)
    test_pred += tmp_pred.reshape(tmp_pred.shape[0], )
    test['prediction'] = test_pred
    test = test[[key, 'prediction']]
    
    if is_plus:
        y_val += (y_min-1)
        y_pred += (y_min-1)
        test_pred += (y_min-1)
#     model_list.append(model)
    
    # Stack Prediction
    df_pred = train.loc[train[key].isin(val_idx), :][[key, target]].copy()
    df_pred['prediction'] = y_pred
    result_list.append(df_pred)
    
    # Scoring
    err = (y_val - y_pred)
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f'RMSE: {score} | SUM ERROR: {err.sum()}')
    score_list.append(score)
    #========================================================================

cv_score = np.mean(score_list)
logger.info(f'''
#========================================================================
# CV SCORE AVG: {cv_score}
#========================================================================''')

#========================================================================
# Stacking
if is_oof:
    pred_cols = [col for col in df_stack.columns if col.count('pred_')]
    df_stack['prediction'] = df_stack[pred_cols].mean(axis=1)
    df_stack.drop(pred_cols, axis=1, inplace=True)
    df_stack.reset_index(inplace=True)
    df_stack = pd.concat([df_stack, test], axis=0, ignore_index=True)
    print(f"DF Stack Shape: {df_stack.shape}")    
else:
    test_pred /= fold
    test['prediction'] = test_pred
    stack_test = test[[key, 'prediction']]
    result_list.append(stack_test)
    df_pred = pd.concat(result_list, axis=0, ignore_index=True).drop(target, axis=1)
    df_stack = base.merge(df_pred, how='inner', on=key)
    print(f"Stacking Shape: {df_stack.shape}")
    del df_pred
    gc.collect()

utils.to_pkl_gzip(obj=df_stack, path=f'../stack/{start_time[4:12]}_elo_NN_stack_E{set_no+1}_row{len(train)}_outpart{out_part}_{len(use_cols)}feat_const{const_cnt}_lr{learning_rate}_batch{batch_size}_epoch{N_EPOCHS}_CV{cv_score}')
#========================================================================

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train on 34048 samples, validate on 6810 samples
Epoch 1/30
 - 3s - loss: 0.9972 - RMSE: 0.9972 - val_loss: 0.9794 - val_RMSE: 0.9794
Epoch 2/30
 - 2s - loss: 0.9726 - RMSE: 0.9726 - val_loss: 0.9671 - val_RMSE: 0.9671
Epoch 3/30
 - 2s - loss: 0.9635 - RMSE: 0.9635 - val_loss: 0.9704 - val_RMSE: 0.9704
Epoch 4/30
 - 2s - loss: 0.9571 - RMSE: 0.9571 - val_loss: 0.9843 - val_RMSE: 0.9843
Epoch 5/30
 - 2s - loss: 0.9500 - RMSE: 0.9500 - val_loss: 0.9646 - val_RMSE: 0.9646
Epoch 6/30
 - 2s - loss: 0.9419 - RMSE: 0.9419 - val_loss: 0.9815 - val_RMSE: 0.9815
Epoch 7/30
 - 2s - loss: 0.9378 - RMSE: 0.9378 - val_loss: 0.9648 - val_RMSE: 0.9648
Epoch 8/30
 - 2s - loss: 0.9282 - RMSE: 0.9282 - val_loss: 0.9815 - val_RMSE: 0.9815
Epoch 9/30
 - 2s - loss: 0.9208 - RMSE: 0.9208 - val_loss: 0.9797 - val_RMSE: 0.9797
Epoch 10/30
 - 2s - loss: 0.9105 - RMSE: 0.9105 - val_loss: 0.9815 - val_RMSE: 0.9815
RMSE: 1.4805

2019-02-11 15:48:32,258 utils 125 [INFO]    [<module>] 
#========================================================================
# CV SCORE AVG: 1.391741201833194
#======================================================================== 


RMSE: 1.2822457045883002 | SUM ERROR: -369.3871130398637
Stacking Shape: (65056, 5)
